<p style="text-align: center">
<img src="../../assets/images/dtlogo.png" alt="Duckietown" width="50%">
</p>

A *state* describes the condition of our robot at a specific time. This may include information about where the robot is, or other factors related to the environment that may affect the robot. When we develop a robotics algorithm, we are responsible to decide the information that we wish to include in the state. In other words, the definition of state depends on the task that we are trying to solve. Ideally, we want a state to exhibit the following properties:

1. Markov property: future state is independent of the past given the present:


$$x_{t+1} = f(x_t, x_{t-1}, \dots, x_0; u_t, \dots, u_0) = f(x_t; u_t)$$


2. A minimally sufficient statistic for the task (i.e., it contains only the information that we need to solve the task)
3. Permits efficient computation
4. Generalizable

An example of a state representation that is often used in robotics is the *pose*. A *pose* represents the location of the robot in the world as well as its orientation (i.e., what direction is the robot facing towards). Since we live in a 3-dimensional (3D) world, we represent the location of the robot as its $(x,y,z)$ coordinate in the world. We can represent orientation in terms of its roll, pitch, and yaw angles (also known as Euler angle representation). For example, the yaw angle $\theta_{yaw}$ will be the angle difference between the longitudinal axis of the robot from the reference frame.

<figure>
  <div style="text-align:center;">
  <img src="../../assets/images/representations/roll_pitch_yaw.png">
  <p>Illustration of roll, pitch, and yaw (source: https://en.wikipedia.org/wiki/Aircraft_principal_axes).</p>
  </div>
</figure>

Note that the location and the orientation of the robot can be defined either in the global coordinate frame (i.e., relative to where the origin is) or relative to any other coordinate frames. Combining location and orientation, we can write the pose $q$ in a vector/matrix form:

$$
q = 
\begin{bmatrix}
x & y & z & \theta_{roll} & \theta_{pitch} & \theta_{yaw}
\end{bmatrix}
$$

The pose $q$ can also be represented in the form of a homogeneous transformation matrix, sinces poses are members of the Matrix Lie Group called the **S**pecial **E**uclidean group $SE(3)$:

$$
SE(3) = \{T = 
\begin{bmatrix}
R & r \\
0^T & 1
\end{bmatrix}
\in \mathrm{R}^{4 \times 4} | R \in SO(3), r \in \mathrm{R}^{3}
\},
$$

where $R$ denotes the rotation matrix, and $r$ denotes the translation component. This matrix $T$ also has a nice property where 

$$
T^{-1} = 
\begin{bmatrix}
R^T & -R^Tr \\
0 & 1
\end{bmatrix}
$$

One of the reasons to represent the pose in this form is because it allows us to conveniently move between frames (see later in this activity). 

Although we have been discussing the pose in $SE(3)$, since we are using a mobile robot and we assume the world is flat, we can simplify the pose of our robot to only its $(x,y)$ coordinate and its yaw (or orientation) angle $\theta$. This is because, in addition to the assumption that the robot is always located at a constant elevation, we can also safely assume roll and pitch angles to always be constant since the robot always touches the ground. So in this case, the pose is simply $
q = 
\begin{bmatrix}
x & y & \theta
\end{bmatrix}
$, which we can also write in $SE(2)$:

$$
SE(2) = \{T = 
\begin{bmatrix}
R & r \\
0^T & 1
\end{bmatrix}
\in \mathrm{R}^{3 \times 3} | R \in SO(2), r \in \mathrm{R}^{2}
\},
$$

where

$$
\small
R = 
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \\
\end{bmatrix}
\ \ \ \
r = 
\begin{bmatrix}
x \\
y \\
\end{bmatrix}
$$

So, we have:

$$
T = 
\begin{bmatrix}
\cos(\theta) & -\sin(\theta) & x \\
\sin(\theta) & \cos(\theta) & y \\
0 & 0 & 1
\end{bmatrix}
$$

Let's first make sure that we understand how to represent a pose by doing the exercise below.

**EXAMPLE: representing pose in $SE(n)$**

<figure>
  <div style="text-align:center;">
  <img src="../../assets/images/representations/pose_exercise.png">
  </div>
</figure>

**Question:**

The figure above illustrates the location of the robot (the orange dot) and its orientation, where the angle is measured from the $x$-axis (assume the angle to have a positive measure if the rotation is counterclockwise). How do we write the pose in $SE(2)$?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Type your answer here
# Tip 1: always express your angles in radians!
# Tip 2: you can go from degrees to radians through the np.deg2rad() function

theta = None

p = np.array([
    [None, None, None],
    [None, None, None],
    [None, None, None]
])

print(theta)
print(p)

(You can find the solutions to this and other questions in the [solutions file](../01-Representations/solutions_pose_representation.ipynb) in this folder. Give it a honest shot yourself before peeking!) 

# II: Moving between frames

It is often useful to convert a pose from one frame to another one, since we may have multiple coordinate frames used in the description of the same problem. Common examples are robot pose with respect to robot or world frame, or pose of an obtacle with respect to the sensor that perceives it or the robot frame. 

As mentioned in the previous section, representing the pose in $SE(n)$ allows us to conveniently move between frames. We can do this by multiplying these transformation matrices together. For example, consider two different poses $a$ and $b$. If we know the pose $a$ in the origin frame $o$ (i.e., $p^o_a$) and the pose $b$ in the frame of $a$ (i.e., $p^a_b$), we can then compute the pose $b$ relative to the origin frame $o$ using the following relation:

$$p^o_b = p^o_a \cdot p^a_b,$$

where $p^o_b$, $p^o_a$, and $p^a_b$ are represented in $SE(n)$. Similarly, if we are given $p^a_o$ and $p^o_b$, we can compute $p^a_b$ via:

$$p^a_b = p^a_o \cdot p^o_b$$

Generally, we can combine these transformations together like the following:

$$p^a_f = p^a_b \cdot p^b_c \cdot p^c_d \cdot p^d_e \cdot p^e_f$$

In addition, another handy relation to know is that $p^a_b = (p^b_a)^{-1}$. 

Let's now take a look at some examples.

**EXAMPLE: moving between frames (from robot frame to map frame)**

Your Duckiebot is on a critical recognition mission: identify and place road obstacles on the map. 

During its mission, your Duckiebot has GPS support, and it knows its own position in the global (map) frame. It encounters its first obstacle at position $x = 2$ m and $y = 0.4$ m, and orientation $\theta$ = 110 degrees. The obstacle itself is at 30cm at 50 degrees (counter-clockwise) from the Duckiebot. Where is the location of the obstacle in the map (i.e., global) frame?

<figure>
  <div style="text-align:center;">
  <img src="../../assets/images/representations/moving_frame_exercise_1.png">
  <p>The orange and blue dots represent the robot and the obstacle, respectively. Determine the position of the blue dot in the global frame given the pose of the Duckiebot in the global from and that of the obstacle with respect to the Duckiebot's frame.</p>
  </div>
</figure>

<!-- We denote our origin frame as $o$, ... as $a$, and ... as $b$. -->

<!-- Compute $p^o_a$ and $p^b_a$, and use them to compute $p^a_b$.  -->

In [ ]:
# Run this cell to initialize the problem
# Tip: be consistent with your units - meters and radians are great choices
# Fact: humans understand degrees better than radians. To avoid confusion, input and output degrees, but do calculations in radians

duckie_pos_g = np.array([2, 0.4])   # Position of Duckiebot in map/global frame (in meters)
duckie_or_g = 110                   # Orientation of Duckiebot in map/global frame (in degrees)
obstacle_dist_to_duckie = 0.3       # Obstacle distance to the Duckiebot
obstacle_angle = 50                 # Obstacle angle with respect to Duckiebot (in degrees)

In [ ]:
# Let's find the answer!

# Step 1. Convert degrees to radians

# Step 2. Transformation matrix from origin (o) to Duckiebot (a)

# Step 3. Transformation matrix from Duckiebot (a) to obstacle (b)

# Step 4. Use the above to compute the transformation from origin (o) to obstacle (b)

# Step 5. Extract the information requested from the transformation matrix

# Write your answer (position of obstacle in global frame) here instead of "None, None" 
obstacle_pos_g = np.array([None, None])    
print(obstacle_pos_g)

Correct result: [1.71809221 0.50260604]

**EXAMPLE: moving between frames (from global frame to robot frame)**

A very concerned duckiecitizen reaches out to you: 

"The sky is falling on our heads!" 

A sizeable fragment of the citizen neighbour's roof fell on the road. You are told it is positioned at $x = 4$ m and $y = -1$ m (in the global reference frame).

This is valuable information, but before making it official by adding it to the map, it needs to be verified. You have a nearby Duckiebot, at $x = 3.5$ m, $y = -1.2$ m, oriented at $\theta = 45$ degrees, that will be tasked to reach the roof fragment. 

What are the coordinates of the obstacle described by the concerned citizen with respect to the Duckiebot's reference frame?

<figure>
  <div style="text-align:center;">
  <img src="../../assets/images/representations/moving_frame_exercise_2.png">
  <p>The orange and blue dots represent the robot and the obstacle, respectively. Determine the position of the blue dot in the robot frame.</p>
  </div>
</figure>

In [ ]:
### Run this cell to initialize the problem
# Tip: remember to be consistent with your units

duckie_pos_g = np.array([3.5, -1.2])   # Position of Duckiebot in global frame (meters)
duckie_or_g = 45                       # Orientation of Duckiebot in global frame (degrees)
obstacle_pos_g = np.array([4, -1])     # Position of obstacle in global frame (meters)

In [ ]:
# Let's send the Duckiebot to the correct place!

# Convert degrees to radians

# Write the transformation matrix from origin (o) to the duckie (a)

# Write the transformation matrix from origin (o) to the obstacle (b)

# Leverage the known properties of transformation matrices and basics of linear algebra to find your answer

# Extract the obstacle position from the transformation matrix

obstacle_x_a = None
obstacle_y_a = None

# Place your answer here instead of None, None (position of obstacle in global frame)
obstacle_pos_r = np.array([obstacle_x_a, obstacle_y_a])    
print(obstacle_pos_r)

Correct result: [ 0.49497475 -0.21213203]

You can now proceed to the [wheel calibration tutorial](../02-Wheel-Calibration/wheels_calibration.ipynb). 

_Credit: [Rey Reza Wiyatno](https://github.com/rrwiyatn)_